In [1]:
import numpy as np
import random
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque

# Set device for acceleration
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

# =========================
# Graph Environment
# =========================
class GraphEnv:
    def __init__(self, A, T, t1=0.1, t2=0.1, t3=10, b=0.5):
        self.A = A.copy()
        self.T = T.copy()
        self.n = len(T)
        self.t1, self.t2, self.t3, self.b = t1, t2, t3, b
        self.done = False

    def clone(self):
        return copy.deepcopy(self)

    def get_neighbors(self, i):
        return set(np.where(self.A[i] == 1)[0])

    def delete_node(self, i):
        self.A[i, :] = 0
        self.A[:, i] = 0
        self.T[i] = 0

    def action_cost(self, a):
        if a == 1: return self.t1 + 4*self.t2 + self.b*self.t3
        if a == 2: return self.t1 + self.t2
        if a == 3: return self.t1 + self.t2
        if a == 4: return self.t1 + 3*self.t2
        if a == 5: return self.t3
        if a == 6: return 3*self.t2 + self.t3
        return 0

    def apply_action(self, action):
        a, nodes = action
        cost = self.action_cost(a)
        
        if a == 1:
            self.T[nodes] = 1
        elif a == 2:
            i, j = nodes
            self.T[j] = 1
            self.delete_node(i)
        elif a in {3, 4, 6}:
            i, _ = nodes
            self.delete_node(i)
        elif a == 5:
            i, j = nodes
            self.A[i, j] = self.A[j, i] = 0

        reward = -cost
        
        self.done = (
                np.all(self.T != -1) # All nodes are either 0 (deleted) or 1 (processed)
                or np.all(self.A.sum(axis=0) == 0)
                or len(self.get_valid_actions()) == 0
            )
        return reward, self.done

    def get_valid_actions(self):
        acts = []
        n = self.n
        for i in range(n):
            if self.T[i] == -1:
                acts.append((1, i))
            if self.T[i] == 1:
                nbrs = self.get_neighbors(i)
                if len(nbrs) == 1:
                    j = next(iter(nbrs))
                    if self.T[j] == -1: acts.append((2, (i, j)))
            if self.T[i] == -1:
                nbrs = self.get_neighbors(i)
                if len(nbrs) == 1:
                    j = next(iter(nbrs))
                    if self.T[j] == 1: acts.append((3, (i, j)))

        for i in range(n):
            for j in range(i+1, n):
                if self.T[i] == -1 and self.T[j] == 1:
                    if self.get_neighbors(i) == self.get_neighbors(j):
                        acts.append((4, (i, j)))
                if self.T[i] == 1 and self.T[j] == 1:
                    if self.A[i, j] == 1: acts.append((5, (i, j)))
                    if self.get_neighbors(i) == self.get_neighbors(j):
                        acts.append((6, (i, j)))
        return acts

    def get_state(self):
        return np.concatenate([self.A.flatten(), self.T])

# =========================
# Shared Action Encoding
# =========================
def encode_action(action, n):
    a, nodes = action
    v = np.zeros(2*n + 6)
    v[a-1] = 1
    if isinstance(nodes, (int, np.integer)):
        v[6 + nodes] = 1
    else:
        i, j = nodes
        v[6 + i] = 1
        v[6 + n + j] = 1
    return v

# =========================
# Q Network
# =========================
class DQN(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        return self.net(x)

# =========================
# Optimized Training
# =========================
def train(env, episodes=300, batch_size=64, gamma=0.99, lr=1e-3):
    n = env.n
    state_dim = n**2 + n
    input_dim = state_dim + 2*n + 6

    q_net = DQN(input_dim).to(device)
    target_net = DQN(input_dim).to(device)
    target_net.load_state_dict(q_net.state_dict())
    
    optimizer = optim.Adam(q_net.parameters(), lr=lr)
    memory = deque(maxlen=20000)
    criterion = nn.MSELoss()

    eps_start, eps_end, eps_decay = 1.0, 0.05, 0.992

    for ep in range(episodes):
        e = env.clone()
        state = e.get_state()
        eps = max(eps_end, eps_start * (eps_decay ** ep))
        ep_reward = 0

        while True:
            acts = e.get_valid_actions()
            if not acts: break

            # Epsilon-Greedy Selection
            if random.random() < eps:
                action = random.choice(acts)
            else:
                q_net.eval()
                with torch.no_grad():
                    # Batch predict all valid actions for current state
                    st_tensor = torch.tensor(state, dtype=torch.float32).repeat(len(acts), 1)
                    ac_tensor = torch.tensor([encode_action(a, n) for a in acts], dtype=torch.float32)
                    inputs = torch.cat([st_tensor, ac_tensor], dim=1).to(device)
                    qs = q_net(inputs)
                    action = acts[torch.argmax(qs).item()]
                q_net.train()

            reward, done = e.apply_action(action)
            next_state = e.get_state()
            next_acts = e.get_valid_actions()
            
            memory.append((state, action, reward, next_state, next_acts, done))
            state = next_state
            ep_reward += reward

            # Vectorized Learning Step
            if len(memory) >= batch_size:
                batch = random.sample(memory, batch_size)
                s_b, a_b, r_b, ns_b, na_b, d_b = zip(*batch)

                # 1. Current Q values
                s_t = torch.tensor(np.stack(s_b), dtype=torch.float32)
                a_t = torch.tensor(np.stack([encode_action(a, n) for a in a_b]), dtype=torch.float32)
                curr_inputs = torch.cat([s_t, a_t], dim=1).to(device)
                curr_q = q_net(curr_inputs).squeeze()

                # 2. Target Q values (Vectorized where possible)
                target_q = torch.zeros(batch_size, device=device)
                with torch.no_grad():
                    for i in range(batch_size):
                        if d_b[i] or not na_b[i]:
                            target_q[i] = r_b[i]
                        else:
                            # Sub-batch for next actions
                            ns_rep = torch.tensor(ns_b[i], dtype=torch.float32).repeat(len(na_b[i]), 1)
                            na_enc = torch.tensor([encode_action(a2, n) for a2 in na_b[i]], dtype=torch.float32)
                            next_inputs = torch.cat([ns_rep, na_enc], dim=1).to(device)
                            max_next_q = torch.max(target_net(next_inputs))
                            target_q[i] = r_b[i] + gamma * max_next_q

                loss = criterion(curr_q, target_q)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            if done: break

        if ep % 10 == 0:
            target_net.load_state_dict(q_net.state_dict())

    return q_net

# =========================
# Fixed Inference
# =========================
def infer_dqn(env, trained_qnet):
    env_copy = env.clone()
    trained_qnet.eval()
    trained_qnet.to(device)
    
    total_cost = 0
    steps = 0
    
    while True:
        valid_actions = env_copy.get_valid_actions()
        if not valid_actions: break

        with torch.no_grad():
            state = env_copy.get_state()
            st_tensor = torch.tensor(state, dtype=torch.float32).repeat(len(valid_actions), 1)
            ac_tensor = torch.tensor([encode_action(a, env_copy.n) for a in valid_actions], dtype=torch.float32)
            inputs = torch.cat([st_tensor, ac_tensor], dim=1).to(device)
            
            qs = trained_qnet(inputs)
            best_idx = torch.argmax(qs).item()
            best_action = valid_actions[best_idx]
        
        reward, done = env_copy.apply_action(best_action)
        # Re-extract the actual cost from the shaped reward for reporting
        actual_step_cost = env_copy.action_cost(best_action[0])
        total_cost += actual_step_cost
        steps += 1
        
        if done: break

    t_processed = int(np.sum(env_copy.T == 1))
    return steps, t_processed, total_cost



In [2]:
def greedy_solve(env):
    env_copy = env.clone()
    steps = 0
    total_cost = 0
    
    while True:
        valid_actions = env_copy.get_valid_actions()
        if not valid_actions:
            break

        # Choose action with minimum immediate cost
        # We use a lambda to sort by the action_cost of the action type (action[0])
        best_action = min(valid_actions, key=lambda act: env_copy.action_cost(act[0]))

        # Track the actual cost before applying any shaping
        actual_cost = env_copy.action_cost(best_action[0])
        total_cost += actual_cost
        
        _, done = env_copy.apply_action(best_action)
        steps += 1
        
        if done:
            break

    t_processed = int(np.sum(env_copy.T == 1))
    return steps, t_processed, total_cost

In [3]:
import networkx as nx

def random_graph(n, p):
    G = nx.erdos_renyi_graph(n, p)
    A = nx.to_numpy_array(G, dtype=int)
    return A

if __name__ == "__main__":
    for i in range(5):
        n = 10
        A = random_graph(n,p=0.3)
        T = np.array([-1] * n)
        env = GraphEnv(A, T, t1=1, t2=1, t3=10,b=0.5)
    
        print("Training start")
        trained_model = train(env, episodes=300)
        print("Training done")
    
        dqn_steps, dqn_ones, dqn_cost = infer_dqn(env, trained_model)
        greedy_steps, greedy_ones, greedy_cost = greedy_solve(env)
    
        print("Results")
        print(f"DQN     Time: {dqn_steps}, Emitter: {dqn_ones}, cost: {dqn_cost}")
        print(f"Greedy  Time: {greedy_steps}, Emitter: {greedy_ones}, cost: {greedy_cost}")


Training start


/tmp/ipykernel_163784/2347620496.py:168: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /build/python-pytorch/src/pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  ac_tensor = torch.tensor([encode_action(a, n) for a in acts], dtype=torch.float32)


Training done
Results
DQN     Time: 10, Emitter: 4, cost: 54.0
Greedy  Time: 10, Emitter: 5, cost: 60.0
Training start
Training done
Results
DQN     Time: 10, Emitter: 10, cost: 100.0
Greedy  Time: 10, Emitter: 10, cost: 100.0
Training start
Training done
Results
DQN     Time: 10, Emitter: 9, cost: 92.0
Greedy  Time: 10, Emitter: 9, cost: 92.0
Training start
Training done
Results
DQN     Time: 11, Emitter: 5, cost: 72.0
Greedy  Time: 10, Emitter: 7, cost: 76.0
Training start
Training done
Results
DQN     Time: 10, Emitter: 9, cost: 92.0
Greedy  Time: 10, Emitter: 9, cost: 92.0
